<a href="https://colab.research.google.com/github/rukmals/ColabProject/blob/master/SASNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

In [2]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), input_shape=(150, 150, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 36992)             0

In [6]:
import torch

In [14]:
layer = tf.keras.layers.Conv2D(32, (3,3), input_shape=(150, 150, 3), activation='relu')


In [7]:
torch.nn.Conv2d(in_channels = 10, out_channels = 10, kernel_size = (3,3), stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')

Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [15]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
       # self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension
        #self.fc2 = nn.Linear(120, 84)
        #self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
)


In [20]:
from torchvision import models
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, kernel_size = 5,stride = 1, padding = 1)
        self.conv2 = nn.Conv2d(6, 16, kernel_size = 5,stride = 1, padding = 1)
        # an affine operation: y = Wx + b
       # self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension
        #self.fc2 = nn.Linear(120, 84)
        #self.fc3 = nn.Linear(84, 10)



net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
)


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
from torchvision import models
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        vgg = models.vgg16_bn(pretrained=False)
        features = list(vgg.features.children())
        # get each stage of the backbone
        self.features1 = nn.Sequential(*features[0:6])
        self.features2 = nn.Sequential(*features[6:13])
        self.features3 = nn.Sequential(*features[13:23])
        self.features4 = nn.Sequential(*features[23:33])
        self.features5 = nn.Sequential(*features[33:43])

        self.de_pred5 = nn.Sequential(
            nn.Conv2d(512, 1024, 3),
            nn.Conv2d(1024, 512, 3),
        )
    # the forward process
    def forward(self, x):
        size = x.size()
        x1 = self.features1(x)
        x2 = self.features2(x1)
        x3 = self.features3(x2)
        x4 = self.features4(x3)
        x5 = self.features5(x4)
        # begining of decoding
        x = self.de_pred5(x5)

        return x




net = Net()
print(net)

Net(
  (features1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
  )
  (features2): Sequential(
    (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU(inplace=True)
    (4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
  )
  (features3): Sequential(
    (0): MaxPool2d(kernel_size=2, stride=2, paddin

In [11]:
from torchvision import models

vgg = models.vgg16_bn(pretrained=False)

vgg

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [5]:
import tensorflow as tf


In [20]:
vgg = tf.keras.applications.VGG16(
    include_top=False, input_tensor=None,
    input_shape=None, pooling=None,
    classifier_activation='softmax'
)

In [27]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [28]:
block5_conv3 = vgg.get_layer("block5_conv3")

In [25]:
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Input, Concatenate


def MCNN(input_shape=None):
    inputs = Input(shape=input_shape)

    # column 1
    column_1 = Conv2D(16, (9, 9), padding='same', activation='relu')(inputs)
    column_1 = MaxPooling2D(2)(column_1)
    column_1 = (column_1)
    column_1 = Conv2D(32, (7, 7), padding='same', activation='relu')(column_1)
    column_1 = MaxPooling2D(2)(column_1)
    column_1 = Conv2D(16, (7, 7), padding='same', activation='relu')(column_1)
    column_1 = Conv2D(8, (7, 7), padding='same', activation='relu')(column_1)

    # column 2
    column_2 = Conv2D(20, (7, 7), padding='same', activation='relu')(inputs)
    column_2 = MaxPooling2D(2)(column_2)
    column_2 = (column_2)
    column_2 = Conv2D(40, (5, 5), padding='same', activation='relu')(column_2)
    column_2 = MaxPooling2D(2)(column_2)
    column_2 = Conv2D(20, (5, 5), padding='same', activation='relu')(column_2)
    column_2 = Conv2D(10, (5, 5), padding='same', activation='relu')(column_2)

    # column 3
    column_3 = Conv2D(24, (5, 5), padding='same', activation='relu')(inputs)
    column_3 = MaxPooling2D(2)(column_3)
    column_3 = (column_3)
    column_3 = Conv2D(48, (3, 3), padding='same', activation='relu')(column_3)
    column_3 = MaxPooling2D(2)(column_3)
    column_3 = Conv2D(24, (3, 3), padding='same', activation='relu')(column_3)
    column_3 = Conv2D(12, (3, 3), padding='same', activation='relu')(column_3)

    # merge feature map of 3 columns in last dimension
    merges = Concatenate(axis=-1)([column_1, column_2, column_3])
    # density map
    density_map = Conv2D(1, (1, 1), padding='same')(merges)

    model = Model(inputs=inputs, outputs=density_map)
    return model


In [2]:
input_shape = (None, None, 1)
model = MCNN(input_shape)

In [5]:
from keras.optimizers import Adam

adam = Adam(lr=1e-4)
model.compile(loss='mse', optimizer=adam)

In [6]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 1 1312        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, None, None, 2 1000        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, None, None, 2 624         input_1[0][0]                    
______________________________________________________________________________________________

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
model = Sequential()


model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))


model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))


model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))


model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))


model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))





In [10]:
from keras.models import Model
from keras.layers import Conv2D, MaxPool2D, Input, Concatenate

def SASNet(input_shape=None):   
    
    inputs = Input(shape=input_shape)

    # column 1
    feature_1 = Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu")(inputs)
    feature_1 = Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu")(feature_1)
    x_1 = (feature_1)

    feature_2 = MaxPool2D(pool_size=(2,2),strides=(2,2))(feature_1)
    feature_2 = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(feature_1)
    feature_2 = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(feature_1)
    x_2 = (feature_2)


    feature_3 = MaxPool2D(pool_size=(2,2),strides=(2,2))(feature_2)
    feature_3 =Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(feature_3)
    feature_3 =Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(feature_3)
    feature_3 =Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(feature_3)
    x_3 = (feature_3)


    feature_4 =MaxPool2D(pool_size=(2,2),strides=(2,2))(feature_3)
    feature_4 =Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(feature_4)
    feature_4 =Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(feature_4)
    feature_4 =Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(feature_4)
    x_4 = (feature_4)


    feature_5 = MaxPool2D(pool_size=(2,2),strides=(2,2))(feature_4)
    feature_5 =Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(feature_5)
    feature_5 =Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(feature_5)
    feature_5 =Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(feature_5)
    x_5 = (feature_5)


    #decorders

    #5th decorder output
    x5_out = Conv2D(filters=1024, kernel_size=(3,3), padding="same", activation="relu")(x_5)

    #4th decorder
    x4_con = Concatenate(axis=-1)([x_4, x5_out])
    x4_out = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(x4_con)

    #3rd decorder
    x3_con = Concatenate(axis=-1)([x_3, x4_out])
    x3_out = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(x3_con)

    #2nd decoder
    x2_con = Concatenate(axis=-1)([x_2, x3_out])
    x2_out = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(x2_con)

    #1st decoder
    x1_con = Concatenate(axis=-1)([x_1, x2_out])
    x1_out = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(x1_con)



    #confidense head
    #5th
    x5_confi = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(x5_out)
    #4th
    x4_confi = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(x4_out)
    #3rd
    x3_confi = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(x3_out)
    #2nd
    x2_confi = Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu")(x2_out)
    #1st
    x1_confi = Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu")(x1_out)


    #density head
    #5th
    x5_density_1 = Conv2D(filters=512, kernel_size=1, padding="same", activation="relu")(x5_out)
    x5_density_1 = Conv2D(filters=256, kernel_size=1, padding="same", activation="relu")(x5_density_1)

    x5_density_2 = Conv2D(filters=512, kernel_size=1, padding="same", activation="relu")(x5_out)
    x5_density_2 = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(x5_density_2)

    x5_density_3 = Conv2D(filters=512, kernel_size=1, padding="same", activation="relu")(x5_out)
    x5_density_3 = Conv2D(filters=256, kernel_size=5, padding="same", activation="relu")(x5_density_3)


    x5_density_out = Concatenate(axis=-1)([x5_density_1,x5_density_2,x5_density_3])


    #4th
    x4_density_1 = Conv2D(filters=512, kernel_size=1, padding="same", activation="relu")(x4_out)
    x4_density_1 = Conv2D(filters=256, kernel_size=1, padding="same", activation="relu")(x4_density_1)

    x4_density_2 = Conv2D(filters=512, kernel_size=1, padding="same", activation="relu")(x4_out)
    x4_density_2 = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(x4_density_2)

    x4_density_3 = Conv2D(filters=512, kernel_size=1, padding="same", activation="relu")(x4_out)
    x4_density_3 = Conv2D(filters=256, kernel_size=5, padding="same", activation="relu")(x4_density_3)

    x4_density_out = Concatenate(axis=-1)([x4_density_1,x4_density_2,x4_density_3])


    #3rd
    x3_density_1 = Conv2D(filters=512, kernel_size=1, padding="same", activation="relu")(x3_out)
    x3_density_1 = Conv2D(filters=256, kernel_size=1, padding="same", activation="relu")(x3_density_1)

    x3_density_2 = Conv2D(filters=512, kernel_size=1, padding="same", activation="relu")(x3_out)
    x3_density_2 = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(x3_density_2)

    x3_density_3 = Conv2D(filters=512, kernel_size=1, padding="same", activation="relu")(x3_out)
    x3_density_3 = Conv2D(filters=256, kernel_size=5, padding="same", activation="relu")(x3_density_3)


    x3_density_out = Concatenate(axis=-1)([x3_density_1,x3_density_2,x3_density_3])


    #2nd
    x2_density_1 = Conv2D(filters=512, kernel_size=1, padding="same", activation="relu")(x2_out)
    x2_density_1 = Conv2D(filters=256, kernel_size=1, padding="same", activation="relu")(x2_density_1)

    x2_density_2 = Conv2D(filters=512, kernel_size=1, padding="same", activation="relu")(x2_out)
    x2_density_2 = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(x2_density_2)

    x2_density_3 = Conv2D(filters=512, kernel_size=1, padding="same", activation="relu")(x2_out)
    x2_density_3 = Conv2D(filters=256, kernel_size=5, padding="same", activation="relu")(x2_density_3)


    x2_density_out = Concatenate(axis=-1)([x2_density_1,x2_density_2,x2_density_3])


    #1st
    x1_density_1 = Conv2D(filters=512, kernel_size=1, padding="same", activation="relu")(x1_out)
    x1_density_1 = Conv2D(filters=256, kernel_size=1, padding="same", activation="relu")(x1_density_1)

    x1_density_2 = Conv2D(filters=512, kernel_size=1, padding="same", activation="relu")(x1_out)
    x1_density_2 = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(x1_density_2)

    x1_density_3 = Conv2D(filters=512, kernel_size=1, padding="same", activation="relu")(x1_out)
    x1_density_3 = Conv2D(filters=256, kernel_size=5, padding="same", activation="relu")(x1_density_3)

    x1_density_out = Concatenate(axis=-1)([x1_density_1,x1_density_2,x1_density_3])




    confi_map = Concatenate(axis=-1)([x5_confi,x4_confi,x3_confi,x2_confi,x1_confi])
    confidense_density_map = Conv2D(1, (1, 1), padding='same',activation = "sigmoid")(confi_map)


    density_map = Concatenate(axis=-1)([x5_density_out,x4_density_out,x3_density_out,x2_density_out,x1_density_out])
    density_head_density_map = Conv2D(1, (1, 1), padding='same',activation = "softmax")(density_map)


    density_head_density_map*=confidense_density_map

    model = Model(inputs=inputs, outputs=density_head_density_map)

    return model
    
    # merge feature map of 3 columns in last dimension
    #merges = Concatenate(axis=-1)([feature_1, feature_2, feature_3,feature_4,feature_5])
    # density map
    #density_map = Conv2D(1, (1, 1), padding='same')(merges)

    #model = Model(inputs=inputs, outputs=density_map)
    #return model

    #return x5_density_3


 


In [11]:
input_shape = (None, None, 1)
model = SASNet(input_shape)

In [12]:
model

In [15]:
from keras.optimizers import Adam

adam = Adam(lr=1e-4)
model.compile(loss='mse', optimizer=adam)

In [21]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


TRAINING_DIR = '/content/sample_data/crowd_data'
train_datagen = ImageDataGenerator(
    rescale=1 / 255,
    rotation_range=40,
    width_shift_range=.2,
    height_shift_range=.2,
    shear_range=.2,
    zoom_range=.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
train_generator = train_datagen.flow_from_directory(
    TRAINING_DIR,
    batch_size=64,
    target_size=(150, 150)
)


VALIDATION_DIR = '/content/sample_data/crowd_data'
validation_datagen = ImageDataGenerator(
    rescale=1 / 255,
    rotation_range=40,
    width_shift_range=.2,
    height_shift_range=.2,
    shear_range=.2,
    zoom_range=.2,
    horizontal_flip=True,
    fill_mode='nearest'

)
validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    batch_size=64,
    target_size=(150, 150)
)

Found 17 images belonging to 2 classes.
Found 17 images belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5


UnimplementedError: ignored

In [23]:
import torch
x = torch.randn((1,3,28,28))
conv = torch.nn.Conv2d(in_channels=3, out_channels=100, kernel_size=3, padding=1)
conv(x).shape


torch.Size([1, 100, 28, 28])

In [24]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten

model = Sequential()
model.add(Conv2D(filters=100, kernel_size=3, padding='valid', input_shape=(28,28,3)))
model.layers[0].output_shape

(None, 26, 26, 100)